## ViEWS 3: Fatalities descriptives, mapping
#### Notebook to describe the dependent variables for the fatalities project, cm level and pgm levels

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sqlalchemy as sa
from ingester3.config import source_db_path

import viewser
from mpl_toolkits.axes_grid1 import make_axes_locatable
import contextily as ctx

from views_dataviz import color
from views_dataviz.map import utils
#from views_dataviz import color
from views_dataviz.map import mapper, utils
from viewser import Queryset, Column
#from views_dataviz.map.presets import ViewsMap


from ingester3.Country import Country
from ingester3.extensions import *
from ingester3.ViewsMonth import ViewsMonth

In [4]:
#creating the geom code, with additional columns for in africa and in middle east
engine = sa.create_engine(source_db_path)
gdf_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, in_africa, in_me, geom FROM prod.country", 
    engine, 
    geom_col='geom'
)
gdf_master = gdf_master.to_crs(4326)

OperationalError: (psycopg2.OperationalError) SSL error: sslv3 alert certificate expired

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [5]:
#adding our data queryset
queryset = (Queryset("m_fat_description", "country_month")
              #CONFLICT TYPES NOT LOGGED and NOT time lagged 
            .with_column(Column("ged_best_sb", from_table = "ged2_cm", from_column = "ged_sb_best_sum_nokgi")
                         .transform.missing.fill()
                        )
            .with_column(Column("ged_best_ns", from_table = "ged2_cm", from_column = "ged_ns_best_sum_nokgi")
                         .transform.missing.fill()
                        )
            .with_column(Column("ged_best_os", from_table = "ged2_cm", from_column = "ged_os_best_sum_nokgi")
                         .transform.missing.fill()
                        )
            
          # Log transformed sb, ns, os
            .with_column(Column("ln_ged_sb", from_table = "ged2_cm", from_column = "ged_sb_best_sum_nokgi")
                         .transform.ops.ln()
                         .transform.missing.fill()
                        )
            .with_column(Column("ln_ged_ns", from_table = "ged2_cm", from_column = "ged_ns_best_sum_nokgi")
                         .transform.ops.ln()
                         .transform.missing.fill()
                        )
            .with_column(Column("ln_ged_os", from_table = "ged2_cm", from_column = "ged_os_best_sum_nokgi")
                         .transform.ops.ln()
                         .transform.missing.fill()
                        )
            #.with_column(Column('name', from_table='country', from_column= 'name')
            #the above does not seem to want to work 
                         
            .with_theme("fatalities")
                         
            .describe("""Fatalities conflict history, cm level
            Fatalities description 
                
            """)
            
           )

data = queryset.publish().fetch()

 .      o   

In [ ]:
#create double logged values
data['ln2_ged_sb'] = np.log1p(data['ln_ged_sb'])
data['ln2_ged_ns'] = np.log1p(data['ln_ged_ns'])
data['ln2_ged_os'] = np.log1p(data['ln_ged_os'])

month id 121: Jan 1990
month id 506: February 2022

In [ ]:
good_range = list(range(121,507))

#limit data based on range
lim_fat_data = data.loc[good_range,]
lim_fat_sum =  pd.DataFrame(lim_fat_data.groupby('country_id').agg({"ged_best_sb": "sum", 
                                                           "ged_best_ns":"sum", 
                                                           "ged_best_os":"sum"}))
#top 10 state
lim_fat_sum.nlargest(10, 'ged_best_sb')
sb_top_min = lim_fat_sum.nlargest(10, 'ged_best_sb')

#top 10 non-state
lim_fat_sum.nlargest(10, 'ged_best_ns')
ns_top_min = lim_fat_sum.nlargest(10, 'ged_best_ns')

#top 10 one sided
lim_fat_sum.nlargest(10, 'ged_best_os')
os_top_min = lim_fat_sum.nlargest(10, 'ged_best_os')

#for sb
lim_fat_sum.loc[lim_fat_sum['ged_best_sb'] >= sb_top_min.ged_best_sb.min(), 'top10_sb'] = 1 
lim_fat_sum.loc[lim_fat_sum['ged_best_sb'] < sb_top_min.ged_best_sb.min(), 'top10_sb'] = 0 
#for ns
lim_fat_sum.loc[lim_fat_sum['ged_best_ns'] >= ns_top_min.ged_best_ns.min(), 'top10_ns'] = 1 
lim_fat_sum.loc[lim_fat_sum['ged_best_ns'] < ns_top_min.ged_best_ns.min(), 'top10_ns'] = 0 
#for os
lim_fat_sum.loc[lim_fat_sum['ged_best_os'] >= os_top_min.ged_best_os.min(), 'top10_os'] = 1 
lim_fat_sum.loc[lim_fat_sum['ged_best_os'] < os_top_min.ged_best_os.min(), 'top10_os'] = 0 

#a quick clean up of the fat_sum data
lim_fat_sum_clean = lim_fat_sum.drop(['ged_best_sb', 'ged_best_ns', 'ged_best_os'], axis=1)
lim_fat_data_dummy = lim_fat_data.join(lim_fat_sum_clean, how='left')
display(lim_fat_data_dummy)

### fatalities in Africa and Middle east for the months Jan 2018, Jun 2018, Dec 2018, Dec2020
#### i also wanted to add the top 10 countries by fatalities across the whole of our queryset, which is partially global for fatalities

In [7]:
#obtain month id for the months above
print(ViewsMonth.from_year_month(year=2022, month=6))
print(ViewsMonth.from_year_month(year=2022, month=9))
print(ViewsMonth.from_year_month(year=2023, month=4))
print(ViewsMonth.from_year_month(year=2024, month=3))

ViewsMonth(id=510) #=> year:2022, month:6
ViewsMonth(id=513) #=> year:2022, month:9
ViewsMonth(id=520) #=> year:2023, month:4
ViewsMonth(id=531) #=> year:2024, month:3


In [7]:
#I wanted to see which countries will be showing up in our list of top 10
display([Country(i).name for i in list(lim_fat_data_dummy.loc[lim_fat_data_dummy.top10_sb > 0].groupby('country_id').groups.keys())])
display(lim_fat_data_dummy.loc[lim_fat_data_dummy.top10_sb > 0].groupby('country_id').groups.keys())
#display(Country(57).name)
#display(Country(191).name)

['Ethiopia',
 'Sudan',
 'Iraq',
 'Somalia',
 'Sri Lanka',
 'Yemen',
 'Afghanistan',
 'Ethiopia',
 'Syria',
 'India']

dict_keys([57, 59, 60, 120, 121, 124, 133, 191, 220, 223])

# MAPPING

In [8]:
class Mapper2:
    """
    `Map` takes basic properties and allows the user to consecutively add
    layers to the Map object. This makes it possible to prepare mapping
    "presets" at any level of layeredness that can be built on further.
    
    Mapper2 allows for the customizable addition of scaling to the map. 
    -re-add the code for labels later when i can test it

    Attributes
    ----------
    width: Integer value for width in inches.
    height: Integer value for height in inches.
    bbox: List for the bbox per [xmin, xmax, ymin, ymax].
    frame_on: Bool for whether to draw a frame around the map.
    title: Optional default title at matplotlib's default size.
    figure: Optional tuple of (fig, size) to use if you want to plot into an
        already existing fig and ax, rather than making a new one.
    """

    def __init__(
        self,
        width,
        height,
        bbox=None,
        cmap=None,
        frame_on=True,
        title="",  # Default title without customization. (?)
        figure=None,
    ):
        self.width = width
        self.height = height
        self.bbox = bbox  # xmin, xmax, ymin, ymax
        self.cmap = cmap
        if figure is None:
            self.fig, self.ax = plt.subplots(figsize=(self.width, self.height))
        else:
            self.fig, self.ax = figure
        self.texts = []
        self.ax.set_title(title)

        if frame_on:  # Remove axis ticks only.
            self.ax.tick_params(
                top=False,
                bottom=False,
                left=False,
                right=False,
                labelleft=False,
                labelbottom=False,
            )
        else:
            self.ax.axis("off")

        if bbox is not None:
            self.ax.set_xlim((self.bbox[0], self.bbox[1]))
            self.ax.set_ylim((self.bbox[2], self.bbox[3]))

    def add_layer(self, gdf, map_scale=False, map_dictionary=None, cmap=None, **kwargs):
        """Add a geopandas plot to a new layer.

        Parameters
        ----------
        gdf: Geopandas GeoDataFrame to plot.
        cmap: Optional matplotlib colormap object or string reference
            (e.g. "viridis"). Defaults to rainbow if not specified.
        map_scale: set a manual scale for the map. If missing defaults to the Remco procedure. 
        map_dictionary: set manual labels for the map. If missing defaults to Remco procedure
        note preferentially looking for map_dictionary, then map_scale, and then Remco default
        no need to use ticklabel and tickvalue commands as the code is subsumed within Mapper2
        refer to separate code for easy common dictionary creations
        **kwargs: Geopandas `.plot` keyword arguments.
        """
        if "color" in kwargs:
            colormap = None
        else:
            colormap = 'rainbow' if cmap is None else cmap
            if "column" in kwargs:
                if hasattr(self, "cax"):
                    self.cax.remove()
                if "vmin" not in kwargs:
                    self.vmin = gdf[kwargs["column"]].min()
                else:
                    self.vmin = kwargs["vmin"]
                if "vmax" not in kwargs:
                    self.vmax = gdf[kwargs["column"]].max()
                else:
                    self.vmax = kwargs["vmax"]
                    
                if map_dictionary is not None:
                    Mapper2.add_colorbar(self, colormap, min(map_dictionary.values()), max(map_dictionary.values()),
                                          tickparams=map_dictionary)
                else:
                    try:Mapper2.add_colorbar(self, colormap, min(map_scale), max(map_scale))
                    except:Mapper2.add_colorbar(self, colormap, self.vmin, self.vmax)
                
        
        if map_dictionary is not None:
                    self.ax = gdf.plot(ax=self.ax, cmap=colormap, vmin=min(map_dictionary.values()),
                               vmax=max(map_dictionary.values()), **kwargs)
        else:
            try: self.ax = gdf.plot(ax=self.ax, cmap=colormap, vmin=min(map_scale), vmax=max(map_scale), **kwargs)
            except: self.ax = gdf.plot(ax=self.ax, cmap=colormap, **kwargs)
                
        return self
    
    def add_colorbar(
        self,
        cmap,
        vmin,
        vmax,
        location="right",
        size="5%",
        pad=0.1,
        alpha=1,
        labelsize=16,
        tickparams=None,
    ):
        """Add custom colorbar to Map.

        Needed since GeoPandas legend and plot axes do not align, see:
        https://geopandas.readthedocs.io/en/latest/docs/user_guide/mapping.html

        Parameters
        ----------
        cmap: Matplotlib colormap object or string reference (e.g. "viridis").
        vmin: Minimum value of range colorbar.
        vmax: Maximum value of range colorbar.
        location: String for location of colorbar: "top", "bottom", "left"
            or "right".
        size: Size in either string percentage or number of pixels.
        pad: Float for padding between the plot's frame and colorbar.
        alpha: Float for alpha to apply to colorbar.
        labelsize: Integer value for the text size of the ticklabels.
        tickparams: Dictionary containing value-label pairs. For example:
            {0.05: "5%", 0.1: "10%"}
        """
        norm = plt.Normalize(vmin, vmax)
        if isinstance(cmap, str):
            cmap = plt.get_cmap(cmap)
        cmap = color.force_alpha_colormap(cmap=cmap, alpha=alpha)
        scalar_to_rgba = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        divider = make_axes_locatable(self.ax)
        self.cax = divider.append_axes(location, size, pad)
        self.cax.tick_params(labelsize=labelsize)
        
        tickvalues = (
            list(tickparams.values()) if tickparams is not None else None
        )
        self.cbar = plt.colorbar(
            scalar_to_rgba, cax=self.cax, ticks=tickvalues
        )
        if tickparams is not None:
            self.cbar.set_ticklabels(list(tickparams.keys()))
        return self
    
    def save(
        self, path, dpi=200, **kwargs
    ):  # Just some defaults to reduce work.
        """Save Map figure to file.
        Parameters
        ----------
        path: String path, e.g. "./example.png".
        dpi: Integer dots per inch. Increase for higher resolution figures.
        **kwargs: Matplotlib `savefig` keyword arguments.
        """
        self.fig.savefig(path, dpi=dpi, bbox_inches="tight", **kwargs)
        plt.close(self.fig)
    
    def add_views_textbox(self, text, textsize=15):
        """Add ViEWS textbox to figure. Logo and url are hardcoded.
        Parameters
        ----------
        text: String text. Note that newlines can be used, for example:
            "Model A\nr_2021_12_01"
        textsize: Integer for textsize.
        """
        utils.add_textbox_to_ax(self.fig, self.ax, text, textsize)
        return self

In [9]:
def vid2date(i):
    year=str(ViewsMonth(i).year)
    month=str(ViewsMonth(i).month)
    return year+'/'+month

def log1p_dict(i):
    labels = [str(i) for i in i]
    values = list(np.log1p(i))
    return dict(zip(labels, values))

def log2p_dict(i):
    labels = [str(i) for i in i]
    values = list(np.log1p(np.log1p(i)))
    return dict(zip(labels, values))

In [10]:
import os
home = os.path.expanduser("~")
display(home)

'/Users/angli742'

### MAP Creation

In [ ]:
list_of_months = [457, 462, 468, 492]
list_of_fatalities = ['sb', 'ns', 'os']
fat_labels = ['state based', 'non-state', 'one-sided']
fact_dict =dict(zip(list_of_fatalities, fat_labels))

standard_values = [0, 10, 50, 100, 1000, 10000]
small_values = [0, 10, 50, 100, 1000]
ludicrous_values = [0, 10, 50, 100, 1000000000]
hh_values = [0,1,3,10,30,100,300,1000,3000,10000]

standard_dictionary = log1p_dict(standard_values)
small_dictionary = log1p_dict(small_values)
ludicrous_dictionary = log1p_dict(ludicrous_values)
hh_dictionary = log1p_dict(hh_values)

folder='/Dropbox (ViEWS)/ViEWS/Projects/PredictingFatalities/maps/cm_actuals_with_top/'
my_path = home+folder


display(standard_dictionary, small_dictionary, ludicrous_dictionary, hh_dictionary, my_path)

In [ ]:
#choose scale and dictionary below
#master code for primary ln runs, run twice once with standard and once with small scale
gdf= gdf_master.copy()
data = lim_fat_data_dummy.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

scale_for_run = hh_dictionary
scale_for_run_name_string = '_hh_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        top10definition = 'top10_'+ key
        variable_def = 'ln_ged_' + key
        title_def = 'Actual fatalities in ' + str(vid2date(i)) + ', ' + value
        textbox = 'Name: '+ key + '_' + str(i) + '_top\nMonth:' + str(i)
        savefile = 'cm_'+str(i)+key+scale_for_run_name_string
        m=Mapper2(
            width=10,
            height=10,
            frame_on=True,
            title=title_def,
            bbox=[-18.5, 64.0, -35.5, 43.0],
        ).add_layer(
            gdf=gdf.loc[i],
            map_dictionary=scale_for_run,
            cmap='rainbow',
            edgecolor="black",
            linewidth=0.5,
            column=variable_def
        ).add_layer(
            gdf=gdf.loc[gdf[top10definition] > 0].geometry.boundary,
            color='black', 
            linewidth=2.0
        ).add_views_textbox(
            text=textbox,
            textsize=16)
        m.save(my_path+savefile)


In [ ]:
#the double logged scale

dlogged_scale = [0,1,3,10,30,100,300,1000,3000,10000]
dlogged_dictionary_2logs = log2p_dict(dlogged_scale)
display(dlogged_dictionary_2logs)

list_of_months = [457, 462, 468, 492]
list_of_fatalities = ['sb', 'ns', 'os']
fat_labels = ['state based', 'non-state', 'one-sided']
fact_dict =dict(zip(list_of_fatalities, fat_labels))
display(fact_dict)

folder='/Dropbox (ViEWS)/ViEWS/Projects/PredictingFatalities/maps/cm_actuals_with_top/'
display(folder)

my_path = home+folder
display(my_path)

In [ ]:
gdf= gdf_master.copy()
data = lim_fat_data_dummy.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

scale_for_run = dlogged_dictionary_2logs
scale_for_run_name_string = '_double_logged_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        variable_def = 'ln2_ged_' + key
        top10definition = 'top10_'+ key
        title_def = 'Actual fatalities double logged in ' + str(vid2date(i)) + ', ' + value
        textbox = 'Name: '+ key + '_' + str(i) + '_double_logged_with_top10\nMonth:' + str(i)
        savefile = 'cm_'+str(i)+key+scale_for_run_name_string
        m=Mapper2(
            width=10,
            height=10,
            frame_on=True,
            title=title_def,
            bbox=[-18.5, 64.0, -35.5, 43.0],
        ).add_layer(
            gdf=gdf.loc[i],
            map_dictionary=scale_for_run,
            cmap='rainbow',
            edgecolor="black",
            linewidth=0.5,
            column=variable_def
        ).add_layer(
            gdf=gdf.loc[gdf[top10definition] > 0].geometry.boundary,
            color='black', 
            linewidth=2.0
        ).add_views_textbox(
            text=textbox,
            textsize=16)
        m.save(my_path+savefile)

# Actual MAPS for PGM

In [ ]:
queryset = (Queryset("m_fatalities_conflict_pgm", "priogrid_month")
              #CONFLICT TYPES NOT LOGGED and NOT time lagged 
            .with_column(Column("ged_best_sb", from_table = "ged2_pgm", from_column = "ged_sb_best_sum_nokgi")
                         .transform.missing.fill()
                        )
            .with_column(Column("ged_best_ns", from_table = "ged2_pgm", from_column = "ged_ns_best_sum_nokgi")
                         .transform.missing.fill()
                        )
            .with_column(Column("ged_best_os", from_table = "ged2_pgm", from_column = "ged_os_best_sum_nokgi")
                         .transform.missing.fill()
                        )
            
              #log transformed
            .with_column(Column("ln_ged_sb", from_table = "ged2_pgm", from_column = "ged_sb_best_sum_nokgi")
                         .transform.ops.ln()
                         .transform.missing.fill()
                        )
            .with_column(Column("ln_ged_ns", from_table = "ged2_pgm", from_column = "ged_ns_best_sum_nokgi")
                         .transform.ops.ln()
                         .transform.missing.fill()
                        )
            .with_column(Column("ln_ged_os", from_table = "ged2_pgm", from_column = "ged_os_best_sum_nokgi")
                         .transform.ops.ln()
                         .transform.missing.fill()
                        )
                
            #.with_column(Column('name', from_table='country', from_column= 'name')
            #the above does not seem to want to work 
                         
            .with_theme("fatalities")
                         
            .describe("""Fatalities conflict history, pgm level
            Fatalities description 
                
            """)
            
           )

data_master = queryset.publish().fetch()

In [ ]:
data_master['ln2_ged_sb'] = np.log1p(data_master['ln_ged_sb'])
data_master['ln2_ged_ns'] = np.log1p(data_master['ln_ged_ns'])
data_master['ln2_ged_os'] = np.log1p(data_master['ln_ged_os'])

In [ ]:
engine = sa.create_engine(source_db_path)
gdf_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as pg_id, in_africa, in_me, geom FROM prod.priogrid", 
    engine, 
    geom_col='geom'
)
gdf_master = gdf_master.to_crs(4326)


In [ ]:
folder='/Dropbox (ViEWS)/ViEWS/Projects/PredictingFatalities/maps/pgm_actuals/'
my_path = home+folder
display(my_path)

small_values = [0, 10, 50, 100, 1000]
very_small = [0, 10, 50, 100, 250]
hh_values = [0,1,3,10,30,100,300,1000]

small_dictionary = log1p_dict(small_values)
very_small_dictionary = log1p_dict(very_small)
hh_dictionary = log1p_dict(hh_values)

display(small_dictionary, very_small_dictionary, hh_dictionary)

list_of_months = [457, 462, 468, 492]
list_of_fatalities = ['sb', 'ns', 'os']
fat_labels = ['state based', 'non-state', 'one-sided']
fact_dict =dict(zip(list_of_fatalities, fat_labels))
display(fact_dict)

In [ ]:
data_pid = data_master.copy()
gdf = gdf_master.copy()
data_pid = data_pid.join(gdf.set_index("pg_id"))
gdf = gpd.GeoDataFrame(data_pid, geometry="geom")

scale_for_run = hh_dictionary
scale_for_run_name_string = '_hh_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        variable_def = 'ln_ged_' + key
        title_def = 'Actual fatalities in ' + str(vid2date(i)) + ', ' + value
        textbox = 'Name: '+ key + '_' + str(i) + '_top\nMonth:' + str(i)
        savefile = 'pgm_'+str(i)+key+scale_for_run_name_string
        m=Mapper2(
            width=10,
            height=10,
            frame_on=True,
            title=title_def,
            bbox=[-18.5, 64.0, -35.5, 43.0],
        ).add_layer(
            gdf=gdf.loc[i],
            map_dictionary=scale_for_run,
            cmap='rainbow',
            edgecolor="black",
            linewidth=0.5,
            column=variable_def
        ).add_views_textbox(
            text=textbox,
            textsize=16)
        m.save(my_path+savefile)

In [ ]:
#attempt at border creation, times out with run
data_pid = data_master.copy()
gdf = gdf_master.copy()
data_pid = data_pid.join(gdf.set_index("pg_id"))
gdf = gpd.GeoDataFrame(data_pid, geometry="geom")

scale_for_run = hh_dictionary
scale_for_run_name_string = '_hh_scale'

plot=Mapper2(
    width=10,
    height=10,
    frame_on=True,
    title=title_def,
    bbox=[-18.5, 64.0, -35.5, 43.0],
    ).add_layer(
    gdf=gdf.loc[457],
    map_dictionary=scale_for_run,
    cmap='rainbow',
    edgecolor="black",
    linewidth=0.5,
    column='ln_ged_sb')
ax=plot.ax
fg=gdf.plot(ax=ax,edgecolor='gray',linewidth=1.0,facecolor='None')
figure=plot.fig
fontdict={'fontsize':20}
fig=plot.fig


In [ ]:
#the double logged scale

dlogged_scale = [0,1,3,10,30,100,300,1000]
dlogged_dictionary_2logs = log2p_dict(dlogged_scale)
display(dlogged_dictionary_2logs)

list_of_months = [457, 462, 468, 492]
list_of_fatalities = ['sb', 'ns', 'os']
fat_labels = ['state based', 'non-state', 'one-sided']
fact_dict =dict(zip(list_of_fatalities, fat_labels))
display(fact_dict)

folder='/Dropbox (ViEWS)/ViEWS/Projects/PredictingFatalities/maps/pgm_actuals/'
display(folder)

my_path = home+folder
display(my_path)

In [ ]:
data_pid = data_master.copy()
gdf = gdf_master.copy()
data_pid = data_pid.join(gdf.set_index("pg_id"))
gdf = gpd.GeoDataFrame(data_pid, geometry="geom")

scale_for_run = dlogged_dictionary_2logs
scale_for_run_name_string = '_double_logged_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        variable_def = 'ln2_ged_' + key
        title_def = 'Actual double logged fatalities in ' + str(vid2date(i)) + ', ' + value
        textbox = 'Name: '+ key + '_' + str(i) + '_top\nMonth:' + str(i)
        savefile = 'pgm_'+str(i)+key+scale_for_run_name_string
        m=Mapper2(
            width=10,
            height=10,
            frame_on=True,
            title=title_def,
            bbox=[-18.5, 64.0, -35.5, 43.0],
        ).add_layer(
            gdf=gdf.loc[i],
            map_dictionary=scale_for_run,
            cmap='rainbow',
            edgecolor="black",
            linewidth=0.5,
            column=variable_def
        ).add_views_textbox(
            text=textbox,
            textsize=16)
        m.save(my_path+savefile)

In [ ]:
#attempt at border creation, times out with run
plot=Mapper2(
    width=10,
    height=10,
    frame_on=True,
    title=title_def,
    bbox=[-18.5, 64.0, -35.5, 43.0],
    ).add_layer(
    gdf=gdf.loc[457],
    map_dictionary=scale_for_run,
    cmap='rainbow',
    edgecolor="black",
    linewidth=0.5,
    column='ln_ged_sb')
ax=plot.ax
fg=gdf.plot(ax=ax,edgecolor='gray',linewidth=1.0,facecolor='None')
figure=plot.fig
fontdict={'fontsize':20}
fig=plot.fig
